In [18]:
import advent
data = advent.get_char_grid(20)

In [19]:
# For dijkstra, we need the 'adjacent' function as always
# My approach: given a portal name, e.g. 'DE', find coordinates next to that portal
# And another function: given coordinate, find the name of the portal there
from functools import cache

@cache # eh why not :P
def find_portal_coordinates(portal_code: str):
    p, q = portal_code[0], portal_code[1]
    result = []
    # first iterate over outer edges, then the middle
    for y in range(data.shape[0]):
        if data[y, 0] == p and data[y, 1] == q: result.append((y, 2))
        if data[y, data.shape[1]-2] == p and data[y, data.shape[1]-1] == q: result.append((y, data.shape[1]-3))
    for x in range(data.shape[1]):
        if data[0, x] == p and data[1, x] == q: result.append((2, x))
        if data[data.shape[0]-2, x] == p and data[data.shape[0]-1, x] == q: result.append((data.shape[0]-3, x))
    for y in range(2, data.shape[0]-2):
        for x in range(2, data.shape[1]-2):
            if data[y, x] == p and data[y, x+1] == q and data[y,x-1] == '.': result.append((y, x-1))
            if data[y, x] == p and data[y, x+1] == q and data[y,x+2] == '.': result.append((y, x+2))
            if data[y, x] == p and data[y+1, x] == q and data[y-1,x] == '.': result.append((y-1, x))
            if data[y, x] == p and data[y+1, x] == q and data[y+2,x] == '.': result.append((y+2, x))
    return result

@cache # same reason as above, maze doesn't change
def find_portal_name(y, x) -> str | None:
    # Doesn't do edge checks since we assume we're in the maze
    assert data[y, x] == '.'
    if data[y-1, x].isalpha(): return data[y-2, x] + data[y-1, x]
    elif data[y+1, x].isalpha(): return data[y+1, x] + data[y+2, x]
    elif data[y, x-1].isalpha(): return data[y, x-2] + data[y, x-1]
    elif data[y, x+1].isalpha(): return data[y, x+1] + data[y, x+2]
    return None

#print(find_portal_coordinates('BC'), find_portal_name(6, 9), find_portal_name(3, 10))


In [22]:
from advent.maze import solve_maze

start = find_portal_coordinates('AA')[0]
end = find_portal_coordinates('ZZ')[0]
is_target = lambda coord: coord == end

def adjacent(coord):
    y, x = coord
    for offset_x, offset_y in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        new_y, new_x = y + offset_y, x + offset_x
        if data[new_y, new_x] == '.':
            yield (new_y, new_x), 1
    portal_name = find_portal_name(y, x)
    if portal_name:
        for new_y, new_x in find_portal_coordinates(portal_name):
            if new_y == y and new_x == x: continue
            yield (new_y, new_x), 1

# Should include (8, 2) for the small test input
#print(list(adjacent((6, 9))))

path = solve_maze(start, is_target, adjacent)


  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 628


In [23]:
# Part 2


def is_inner(coord):
    # Given a coord (that is '.'), is it on the outer edge? if so return false
    y, x = coord
    return 2 < y < data.shape[0] - 3 and 2 < x < data.shape[1] - 3

# coords are now ((y, x), level)
start_part2 = find_portal_coordinates('AA')[0], 0
end_part2 = find_portal_coordinates('ZZ')[0], 0
is_target_part2 = lambda coord: coord == end_part2

def adjacent_part2(coord):
    (y, x), level = coord
    assert data[y, x] == '.' and level >= 0, coord
    for offset_x, offset_y in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        new_y, new_x = y + offset_y, x + offset_x
        if data[new_y, new_x] == '.':
            yield ((new_y, new_x), level), 1
    portal_name = find_portal_name(y, x)
    if portal_name:
        for new_y, new_x in find_portal_coordinates(portal_name):
            if new_y == y and new_x == x: continue
            if is_inner((y, x)): # we were on the inside so we are going down a level
                yield ((new_y, new_x), level+1), 1
            elif level > 0: # We were on outside so we are going up a level
                # however this is only allowed if we are not on level 0
                yield ((new_y, new_x), level-1), 1

path = solve_maze(start_part2, is_target_part2, adjacent_part2)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 7506
